## La Liga Big 3 (2014–2025): Seasonal Goals & Expected Goals (xG) Comparison
- Real Madrid, Barcelona, and Atletico Madrid — season-by-season trends in Goals For (GF) and Expected Goals For (xGF), with simple interpretations.

### Step A. Imports & Paths

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

df = pd.read_csv('la_liga_2014-2025_match_info.csv')

print(df.head())

   match_id                 date home_team            away_team  home_goals  \
0      5826  2014-08-23 18:00:00    Malaga        Athletic Club           1   
1      5827  2014-08-23 20:00:00   Sevilla             Valencia           1   
2      5828  2014-08-23 20:00:00   Granada  Deportivo La Coruna           2   
3      5829  2014-08-23 22:00:00   Almeria             Espanyol           1   
4      5830  2014-08-24 18:00:00     Eibar        Real Sociedad           1   

   away_goals  home_xg  away_xg  home_shots  away_shots  home_sot  away_sot  \
0           0     1.32     1.14          12          12         3         5   
1           1     1.17     1.75          12          11         3         1   
2           1     0.55     0.38          10           8         3         1   
3           1     0.98     0.40          19          12         6         2   
4           0     0.47     0.98          12          19         5         4   

   home_deep  away_deep  home_ppda  away_ppda  hom

In [23]:
# Step A — Imports & Paths
# EN: Prepare libraries and set the CSV path.


### Step B. Load & Create Season Labels

In [8]:
df = pd.read_csv('la_liga_2014-2025_match_info.csv')
df["date"] = pd.to_datetime(df["date"], errors="raise")

# Create 'season' if missing (La Liga runs Aug–May)
def make_season(y, m): 
    return f"{y}-{y+1}" if m >= 8 else f"{y-1}-{y}"

if "season" not in df.columns:
    df["season"] = df.apply(lambda r: make_season(r["date"].year, r["date"].month), axis=1)

print(df[["date","home_team","away_team","home_goals","away_goals","home_xg","away_xg"]].head())


                 date home_team            away_team  home_goals  away_goals  \
0 2014-08-23 18:00:00    Malaga        Athletic Club           1           0   
1 2014-08-23 20:00:00   Sevilla             Valencia           1           1   
2 2014-08-23 20:00:00   Granada  Deportivo La Coruna           2           1   
3 2014-08-23 22:00:00   Almeria             Espanyol           1           1   
4 2014-08-24 18:00:00     Eibar        Real Sociedad           1           0   

   home_xg  away_xg  
0     1.32     1.14  
1     1.17     1.75  
2     0.55     0.38  
3     0.98     0.40  
4     0.47     0.98  


In [24]:
# Step B — Load & Season Labels
# EN: Load matches and create a 'season' label (Aug–May) from the match date if missing.
# Columns used: date (parse to datetime); if season missing, build from date.


### Step C. Build team-match rows (Home & Away to one “team” table)

In [10]:
# Convert each match into two rows: one for home team, one for away team
home = df[["date","season","home_team","home_goals","away_goals","home_xg","away_xg"]].copy()
home.columns = ["date","season","team","gf","ga","xgf","xga"]

away = df[["date","season","away_team","away_goals","home_goals","away_xg","home_xg"]].copy()
away.columns = ["date","season","team","gf","ga","xgf","xga"]

tm = pd.concat([home, away], ignore_index=True)
tm.sort_values(["team","date"], inplace=True)

# Points per match (3/1/0) for later (optional)
tm["pts"] = np.where(tm["gf"]>tm["ga"], 3, np.where(tm["gf"]==tm["ga"], 1, 0))

print(tm.head())


                    date     season    team  gf  ga   xgf   xga  pts
4947 2016-08-22 00:15:00  2016-2017  Alaves   1   1  0.03  4.18    1
777  2016-08-28 20:15:00  2016-2017  Alaves   0   0  1.12  0.28    1
4965 2016-09-10 22:30:00  2016-2017  Alaves   2   1  0.29  1.35    3
799  2016-09-19 22:45:00  2016-2017  Alaves   0   0  0.46  0.67    1
4989 2016-09-23 00:00:00  2016-2017  Alaves   1   2  0.26  1.42    0


In [25]:
# Step C — Team-Match Table (Home/Away -> Team rows)
# EN: Convert each match into two team rows (home team & away team) with GF/GA and xGF/xGA.
#     Also compute points per match (3 win / 1 draw / 0 loss) for later use.
# Columns used: date, season, home_team, away_team, home_goals, away_goals, home_xg, away_xg
# Derived: team, gf, ga, xgf, xga, pts

### Step D. Filter the “Big 3” & Aggregate by Season

In [12]:
# Team name robustness (handles minor variants)
big3_alias = {
    "Real Madrid": ["Real Madrid"],
    "Barcelona":   ["Barcelona","FC Barcelona"],
    "Atletico Madrid": ["Atletico Madrid","Atlético Madrid"]
}

def is_big3(name):
    return any(name in v for v in big3_alias.values()) or \
           any(any(alias==name for alias in v) for v in big3_alias.values())

# Safer: map to canonical names
def canonical(name):
    for k, aliases in big3_alias.items():
        if name in aliases:
            return k
    return name

tm["team_std"] = tm["team"].apply(canonical)
big3 = tm[tm["team_std"].isin(["Real Madrid","Barcelona","Atletico Madrid"])].copy()

# Season-level per-team averages
season_team = (big3
               .groupby(["season","team_std"])
               .agg(
                   matches=("team_std","size"),
                   GF=("gf","sum"), GA=("ga","sum"),
                   xGF=("xgf","sum"), xGA=("xga","sum"),
                   Pts=("pts","sum")
               )
               .reset_index())

# Per-match metrics
season_team["GF_per_match"]  = season_team["GF"]/season_team["matches"]
season_team["GA_per_match"]  = season_team["GA"]/season_team["matches"]
season_team["xGF_per_match"] = season_team["xGF"]/season_team["matches"]
season_team["xGA_per_match"] = season_team["xGA"]/season_team["matches"]
season_team["PPG"]           = season_team["Pts"]/season_team["matches"]  # points per game

season_team.head()


,season,team_std,matches,GF,GA,xGF,xGA,Pts,GF_per_match,GA_per_match,xGF_per_match,xGA_per_match,PPG
0,2014-2015,Atletico Madrid,38,67,29,57.05,29.08,78,1.763158,0.763158,1.501316,0.765263,2.052632
1,2014-2015,Barcelona,38,110,21,102.98,28.47,94,2.894737,0.552632,2.710000,0.749211,2.473684
2,2014-2015,Real Madrid,38,118,38,95.76,42.58,92,3.105263,1.000000,2.520000,1.120526,2.421053
3,2015-2016,Atletico Madrid,38,63,18,54.94,27.79,88,1.657895,0.473684,1.445789,0.731316,2.315789
4,2015-2016,Barcelona,38,112,29,113.59,34.05,91,2.947368,0.763158,2.989211,0.896053,2.394737


In [26]:
# Step D — Filter Big 3 & Aggregate by Season
# EN: Keep only Real Madrid, Barcelona, Atletico Madrid (canonical names) and aggregate by season.
#     Sum totals (GF/GA/xGF/xGA/points) and compute per-match metrics.
# Columns used: season, team (mapped to team_std), gf, ga, xgf, xga, pts
# Derived: matches, GF_per_match, GA_per_match, xGF_per_match, xGA_per_match, PPG

### Step E. Plot 1: Seasonal GF per match (line)

In [14]:
import plotly.express as px

# EN: Interactive line with hover (Season, Team, GF per match)
# KO: 인터랙티브 라인 (시즌, 팀, 경기당 득점 표시)
gf_long = season_team[["season","team_std","GF_per_match"]].copy()

fig_gf = px.line(
    gf_long, x="season", y="GF_per_match", color="team_std",
    markers=True, title="Seasonal Goals For per Match — Big 3",
    labels={"season":"Season","GF_per_match":"GF per match","team_std":"Team"},
    hover_data={"season":True,"team_std":True,"GF_per_match":":.2f"}
)
fig_gf.update_layout(legend_title_text="Team", xaxis_tickangle=45)
fig_gf.show()


In [27]:
# Step E — Plot: Seasonal GF per Match (Big 3)
# EN: Line chart of goals for per match by season for the Big 3.
# Columns used: season_team['season','team_std','GF_per_match']
# 사용 컬럼: season_team['season','team_std','GF_per_match']

- Barcelona peaks around 2015–2017, then drops sharply by 2018–2020, and rises again by 2024–2025.
- Real Madrid dips around 2018–2021, then recovers 2022–2024; slight pullback in 2024–2025.
- Atletico Madrid sits lower than the other two most seasons, with a steady uptick from 2021–2024.

### Step F. Plot 2: Seasonal xGF per match (line)

In [16]:
# EN: Interactive line for expected goals (xGF) per match.
# KO: 경기당 기대득점(xGF) 인터랙티브 라인.
xgf_long = season_team[["season","team_std","xGF_per_match"]].copy()

fig_xgf = px.line(
    xgf_long, x="season", y="xGF_per_match", color="team_std",
    markers=True, title="Seasonal Expected Goals For (xGF) per Match — Big 3",
    labels={"season":"Season","xGF_per_match":"xGF per match","team_std":"Team"},
    hover_data={"season":True,"team_std":True,"xGF_per_match":":.2f"}
)
fig_xgf.update_layout(legend_title_text="Team", xaxis_tickangle=45)
fig_xgf.show()


In [17]:
# Step F — Plot: Seasonal xGF per Match (Big 3)
# EN: Line chart of expected goals for per match (xGF) by season for the Big 3.
# KO: 3강의 시즌별 경기당 기대득점(xGF)을 라인 차트로 비교합니다.
# Columns used: season_team['season','team_std','xGF_per_match']
# 사용 컬럼: season_team['season','team_std','xGF_per_match']

- Barcelona leads xGF in mid-2010s, then declines to 2019–2021, rebounds afterwards.
- Real Madrid shows a trough near 2020–2021, then climbs again 2021–2025.
- Atletico has the lowest xGF overall, but trends upward from 2021–2025.

## La Liga Big 3 Attack Efficiency Analysis (2014-2025): A Comparison of Goals For vs. Expected Goals For



1. Purpose of the Analysis

- The goal was to move beyond simply counting goals and to quantitatively evaluate the quality and clinical nature of each team's attack. This analysis helps determine whether a team's performance is driven by creating a high volume of chances or by exceptionally good finishing.

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

2. Metrics Used

- GF (Goals For): The total number of goals a team actually scored. 

- xGF (Expected Goals For): The number of goals a team would be expected to score based on the quality (e.g., shot location, angle, type of assist) of the chances they created. 

- Attack Efficiency: The key metric derived from the two above, calculated as: (GF per Match) - (xGF per Match).

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

3. Meaning of the Metrics

- GF represents the raw, final output of an attack.

- xGF represents the quality of the chances created. A high xGF indicates a team is skilled at getting into dangerous scoring positions.


3. Attack Efficiency measures finishing skill:

- A positive value indicates over-performance; the team scored more goals than expected, suggesting excellent finishing, good fortune, or superior shooting talent.

- A negative value indicates under-performance; the team scored fewer goals than expected, suggesting wastefulness, bad luck, or poor shot execution.

- A value near zero suggests that the team's scoring was in line with the quality of chances it created.

#### Step 1: Import Data

In [18]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

#### Step 2: Load and Process Data

In [19]:
# Load data and create season column
df = pd.read_csv('la_liga_2014-2025_match_info.csv')
df["date"] = pd.to_datetime(df["date"], errors="raise")

def make_season(y, m):
    return f"{y}-{y+1}" if m >= 8 else f"{y-1}-{y}"

if "season" not in df.columns:
    df["season"] = df.apply(lambda r: make_season(r["date"].year, r["date"].month), axis=1)

# Transform data into team-centric rows
home = df[["date","season","home_team","home_goals","away_goals","home_xg","away_xg"]].copy()
home.columns = ["date","season","team","gf","ga","xgf","xga"]

away = df[["date","season","away_team","away_goals","home_goals","away_xg","home_xg"]].copy()
away.columns = ["date","season","team","gf","ga","xgf","xga"]

tm = pd.concat([home, away], ignore_index=True)

# Standardize team names and filter for the Big 3
big3_alias = {
    "Real Madrid": ["Real Madrid"],
    "Barcelona":   ["Barcelona","FC Barcelona"],
    "Atletico Madrid": ["Atletico Madrid","Atlético Madrid"]
}
def canonical(name):
    for k, aliases in big3_alias.items():
        if name in aliases:
            return k
    return name

tm["team_std"] = tm["team"].apply(canonical)
big3 = tm[tm["team_std"].isin(["Real Madrid","Barcelona","Atletico Madrid"])].copy()

# Aggregate by season and team
season_team = (big3
               .groupby(["season","team_std"])
               .agg(
                   matches=("team_std","size"),
                   GF=("gf","sum"),
                   xGF=("xgf","sum"),
               )
               .reset_index())

# Calculate per-match metrics
season_team["GF_per_match"]  = season_team["GF"]/season_team["matches"]
season_team["xGF_per_match"] = season_team["xGF"]/season_team["matches"]

print("Data processing complete. The 'season_team' DataFrame is ready.")
season_team.head()

Data processing complete. The 'season_team' DataFrame is ready.


,season,team_std,matches,GF,xGF,GF_per_match,xGF_per_match
0,2014-2015,Atletico Madrid,38,67,57.05,1.763158,1.501316
1,2014-2015,Barcelona,38,110,102.98,2.894737,2.710000
2,2014-2015,Real Madrid,38,118,95.76,3.105263,2.520000
3,2015-2016,Atletico Madrid,38,63,54.94,1.657895,1.445789
4,2015-2016,Barcelona,38,112,113.59,2.947368,2.989211


#### Step 3: Calculate Attack Efficiency

In [20]:
season_team['attack_efficiency'] = season_team['GF_per_match'] - season_team['xGF_per_match']
print("Attack Efficiency calculated.")
season_team[['season', 'team_std', 'attack_efficiency']].head()

Attack Efficiency calculated.


,season,team_std,attack_efficiency
0,2014-2015,Atletico Madrid,0.261842
1,2014-2015,Barcelona,0.184737
2,2014-2015,Real Madrid,0.585263
3,2015-2016,Atletico Madrid,0.212105
4,2015-2016,Barcelona,-0.041842


#### Step 4. Visualize the Results

In [21]:
fig = px.bar(
    season_team,
    x="season",
    y="attack_efficiency",
    color="team_std",
    barmode='group',
    title="Big 3 Attack Efficiency (GF per Match vs xGF per Match)",
    labels={
        "season": "Season",
        "attack_efficiency": "Attack Efficiency (GF - xGF)",
        "team_std": "Team"
    },
    hover_data={"attack_efficiency": ":.2f"}
)

# Add a horizontal line at y=0 to easily distinguish over/under-performance
fig.add_hline(y=0, line_width=2, line_dash="dash", line_color="black")

fig.update_layout(legend_title_text="Team", xaxis_tickangle=45)

# In a Jupyter environment, this will display the interactive chart directly in the output.
fig.show()

1. Real Madrid's Peak and Post-Ronaldo Decline:

- Real Madrid demonstrated exceptional over-performance in the 2014-2015 (+0.59) and 2015-2016 (+0.51)  seasons, indicating a world-class ability to convert chances during this period.

- A significant shift occurred in the 2018-2019 season, where they posted a negative efficiency (-0.15) for the first time in this dataset, coinciding with the departure of Cristiano Ronaldo.

- The team recorded its worst finishing performance in the 2024-2025 season with an efficiency of -0.27.

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

2. Barcelona's Volatile Finishing:

- Barcelona reached the highest peak of any team in the 2016-2017 season with an attack efficiency of +0.59, a testament to the clinical nature of the MSN (Messi, Suárez,Neymar) attacking trio.

- However, the team has also shown periods of extreme inefficiency, most notably in the 2022-2023 season where their efficiency plummeted to -0.34, indicating significant struggles in converting quality chances into goals.

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

3. Atlético Madrid's Relative Consistency:

- Atlético Madrid generally performed closer to their expected values compared to the other two giants, showing consistent over-performance from 2014 to 2018.

- Their most significant period of under-performance occurred in the 2019-2020 season, with an efficiency score of -0.20.
